In [1]:
import numpy as np
from astropy.table import Table
from astropy import table
from astroquery.vizier import Vizier
from astroquery.gaia import Gaia
from astroquery.mast import Catalogs
from astropy.coordinates import SkyCoord
from astropy.io.votable import parse
from astropy.constants import G, M_sun, pc, R_sun, h, c, k_B
# from dust_extinction.parameter_averages import G23
import extinction
import astropy.units as u
from scipy.optimize import curve_fit
import WD_models
import os.path
from os import listdir
import matplotlib.pyplot as plt
from csv import writer
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

figwidth_single = 89/25.4  # [inch]
figwidth_double = 183/25.4  # [inch]
%matplotlib tk

# Fitting SEDs

## Functions

In [2]:
## filter zero points, effective wavelengths, column names

## f = f0 * 10^(-0.4*(m-m0)) (erg/s/cm^2/AA) 

## wise_f0 = (np.array([309.540,171.787,31.674]) * u.Jy.to(u.erg/u.s/u.cm**2/u.Hz) * u.erg/u.s/u.cm**2/u.Hz * c.cgs / (np.array([33526,46030,115608]) * u.AA)**2).to(u.erg/u.s/u.cm**2/u.AA)

## gaia_f0 = np.array([3.009167e-21,1.346109e-21,1.638483e-21]) * (u.W/u.m**2/u.nm).to(u.erg/u.s/u.cm**2/u.AA)

# two_mass_f0 = np.array([3.129e-13,1.133e-13,4.283e-14]) * (u.W / u.cm**2 / u.micron).to(u.erg/u.s/u.cm**2/u.AA)


twomass_zp_table = Table({'band': ['2MASS.J','2MASS.H', '2MASS.Ks'],
                        'col':['Jmag','Hmag','Kmag'],
                        'err_col':['e_Jmag','e_Hmag','e_Kmag'],
                        'lambda_eff':[12350,16620,21590],
                        'f0':[3.129e-10,1.133e-10,4.283e-11],
                        'm0':[0,0,0]})

wise_zp_table = Table({'band': ['WISE.W1','WISE.W2','WISE.W3'],
                       'col':['W1mag','W2mag','W3mag'],
                          'err_col':['e_W1mag','e_W2mag','e_W3mag'],
                          'lambda_eff':[33526,46028,115608],
                                'f0':[8.18e-12,2.42e-12,6.52e-14],
                                'm0':[0,0,0]})

gaia_zp_table = Table({'band': ['GAIA3.Gbp','GAIA3.G','GAIA3.Grp'],
                       'col':['phot_bp_mean_mag','phot_g_mean_mag','phot_rp_mean_mag'],
                       'err_col':['phot_bp_mean_flux_error','phot_g_mean_flux_error','phot_rp_mean_flux_error'],
                       'lambda_eff':[5035,5822,7619],
                       'f0':[4.08e-9,2.5e-9,1.27e-9],
                       'm0':[0,0,0]})
galex_zp_table = Table({'band': ['GALEX.FUV', 'GALEX.NUV'],
                       'col':['fuv_mag','nuv_mag'],
                       'err_col':['fuv_magerr','nuv_magerr'],
                       'lambda_eff':[1548,2303],
                       'f0':[4.6e-8,2.05e-8],
                       'm0':[0,0]})

synt_zp_table = Table({'band': ['Johnson.U', 'Johnson.B','Johnson.V','Johnson.R','Johnson.I',
                             'SDSS.u','SDSS.g','SDSS.r','SDSS.i','SDSS.z'],
                     'lambda_eff':[3551.05,4369.53,5467.57,6695.83,8568.89,3608.04,4671.78,6141.12,7457.89,8922.78],
                    'col':['u_jkc_flux','b_jkc_flux','v_jkc_flux','r_jkc_flux','i_jkc_flux',
                           'u_sdss_flux','g_sdss_flux','r_sdss_flux','i_sdss_flux','z_sdss_flux']})

bands_table = Table({'band': list(gaia_zp_table['band'])+list(wise_zp_table['band'])+list(twomass_zp_table['band'])+list(synt_zp_table['band']) + list(galex_zp_table),
                     'lambda_eff': list(gaia_zp_table['lambda_eff'])+list(wise_zp_table['lambda_eff'])+list(twomass_zp_table['lambda_eff'])+list(synt_zp_table['lambda_eff']) + list(galex_zp_table['lambda_eff'])})


Table.sort(bands_table, 'lambda_eff')

In [24]:
## get photometry

def get_synthetic_photometry(source_table):
    ## query gaia synthetic photometry for SDSS ugriz and Johnson UBVRI, convert to erg/s/cm^2/AA
    id_lst = tuple(source_table['source_id'])
    cols_to_query =','.join(['source_id'] + [c + ',' + c +'_error' for c in synt_zp_table['col']])
    query = 'SELECT ' + cols_to_query + f' FROM gaiadr3.synthetic_photometry_gspc WHERE source_id IN {id_lst}'
    job = Gaia.launch_job(query)
    result = job.get_results()
    for col in synt_zp_table['col']:
        band = synt_zp_table[synt_zp_table['col']==col]['band'][0]
        err_col = col + '_error'
        if band.startswith('Johnson'): ## Johnson fluxes given in W/s/m^2/nm
            result[col] = result[col].to(u.erg/u.s/u.cm**2/u.AA)
            result[err_col] = result[err_col].to(u.erg/u.s/u.cm**2/u.AA)
        elif band.startswith('SDSS'): ## SDSS fluxes given in W/s/m^2/Hz
            wl = synt_zp_table[synt_zp_table['col']==col]['lambda_eff'][0] * u.AA
            result[col] = (result[col].data * u.W / u.m**2 / u.Hz * c / wl.si**2).to(u.erg/u.s/u.cm**2/u.AA)
            result[err_col] = (result[err_col].data * u.W / u.m**2 / u.Hz * c / wl.si**2).to(u.erg/u.s/u.cm**2/u.AA)
        result[col].name = band
        result[err_col].name = band +'_err'
    if 'SOURCE_ID' in result.colnames:
        result.rename_column('SOURCE_ID','source_id')
    return result

def get_gaia_photometry(source_table):
    id_lst = tuple(source_table['source_id'])
    cols_to_query =','.join(['source_id'] + [c for c in gaia_zp_table['col']] + [c.replace('mag','flux') + ',' + c.replace('mag','flux') +'_error' for c in gaia_zp_table['col']])
    query = 'SELECT ' + cols_to_query + f' FROM gaiadr3.gaia_source WHERE source_id IN {id_lst}'
    job = Gaia.launch_job(query)
    result = job.get_results()
    for col in gaia_zp_table['col']:
        band = gaia_zp_table[gaia_zp_table['col']==col]['band'][0]
        err_col = gaia_zp_table[gaia_zp_table['col']==col]['err_col'][0]
        f0 = gaia_zp_table[gaia_zp_table['col']==col]['f0'][0]
        m0 = gaia_zp_table[gaia_zp_table['col']==col]['m0'][0]
        err_over_f = result[col.replace('mag','flux')+'_error'] / result[col.replace('mag','flux')] ## fractional error- the fluxes are in arbitrary units, but the fractional error is the same as in erg/s/cm^2/AA
        mag = result[col]
        result[col] = f0 * 10**(-0.4*(mag-m0))
        result[col].unit = u.erg / u.s / u.cm**2 / u.AA
        result[col].name = band
        result[err_col] = result[band] * err_over_f ## to convert fractional error to absolute error in physical units
        result[err_col].unit = u.erg / u.s / u.cm**2 / u.AA
        result[err_col].name = band + '_err'
    if 'SOURCE_ID' in result.colnames:
        result.rename_column('SOURCE_ID','source_id')
    result.keep_columns(['source_id'] + list(gaia_zp_table['band']) + [band + '_err' for band in gaia_zp_table['band']])
    return result

def get_wise_photometry(source_table):
    coords = SkyCoord(source_table['ra'], source_table['dec'], unit=(u.deg, u.deg), frame='icrs')
    wise = Vizier.query_region(coords, radius=2*u.arcsec,catalog=['II/311'])
    wise = wise[0]
    wise['_q'] = wise['_q'] - 1
    wise['_q'] = source_table['idx'][wise['_q']] 
    wise['_q'].name = 'idx'
    wise.keep_columns(['idx'] + list(wise_zp_table['col']) + list(wise_zp_table['err_col']))
    for col in wise_zp_table['col']:
        band = wise_zp_table[wise_zp_table['col']==col]['band'][0]
        err_col = wise_zp_table[wise_zp_table['col']==col]['err_col'][0]
        f0 = wise_zp_table[wise_zp_table['col']==col]['f0'][0]
        m0 = wise_zp_table[wise_zp_table['col']==col]['m0'][0]
        wise[col] = f0 * 10**(-0.4*(wise[col]-m0))
        wise[col].unit = u.erg / u.s / u.cm**2 / u.AA
        wise[col].name = band
        wise[err_col] = wise[band] * 0.4 * np.log(10) * wise[err_col] ## convert mag error to flux error
        wise[err_col].unit = u.erg / u.s / u.cm**2 / u.AA
        wise[err_col].name = band + '_err'
    for band in wise_zp_table['band']: ## mask out sources with low SNR (only keep sources WISE would flag A or B)
        for i in range(len(wise)):
            if wise[i][band] / wise[i][band + '_err'] < 3:
                wise[i][band] = np.nan
                wise[i][band + '_err'] = np.nan
    for idx in source_table['idx']:
        if idx not in wise['idx']:
            wise.add_row([idx] + [np.nan for col in wise_zp_table['col']] + [np.nan for err_col in wise_zp_table['err_col']])
    wise.sort('idx')
    return wise

def get_twomass_photometry(source_table):
    coords = SkyCoord(source_table['ra'], source_table['dec'], unit=(u.deg, u.deg), frame='icrs')
    twomass = Vizier.query_region(coords, radius=2*u.arcsec,catalog=['II/246'])
    twomass = twomass[0]
    twomass['_q'] = twomass['_q'] - 1
    twomass['_q'] = source_table['idx'][twomass['_q']]
    twomass['_q'].name = 'idx'
    quality_flag = twomass['Qflg']
    twomass.keep_columns(['idx'] + list(twomass_zp_table['col']) + list(twomass_zp_table['err_col']))
    for col in twomass_zp_table['col']:
        band = twomass_zp_table[twomass_zp_table['col']==col]['band'][0]
        err_col = twomass_zp_table[twomass_zp_table['col']==col]['err_col'][0]
        f0 = twomass_zp_table[twomass_zp_table['col']==col]['f0'][0]
        m0 = twomass_zp_table[twomass_zp_table['col']==col]['m0'][0]
        twomass[col] = f0 * 10**(-0.4*(twomass[col]-m0))
        twomass[col].unit = u.erg / u.s / u.cm**2 / u.AA
        twomass[col].name = band
        twomass[err_col] = twomass[band] * 0.4 * np.log(10) * twomass[err_col] ## convert mag error to flux error
        twomass[err_col].unit = u.erg / u.s / u.cm**2 / u.AA
        twomass[err_col].name = band + '_err'
    for i in range(len(twomass)):
        for j in [0,1,2]: ## J,H,Ks bands respectively
            if quality_flag[i][j] != 'A': ## mask out sources according to 2MASS quality flag
                twomass[i][twomass_zp_table['band'][j]] = np.nan
                twomass[i][twomass_zp_table['band'][j] + '_err'] = np.nan
    for idx in source_table['idx']:
        if idx not in twomass['idx']:
            twomass.add_row([idx] + [np.nan for col in twomass_zp_table['col']] + [np.nan for err_col in twomass_zp_table['err_col']])
    twomass.sort('idx')
    return twomass

def get_galex_photometry(source_table):
    coords = SkyCoord(source_table['ra'], source_table['dec'], unit=(u.deg, u.deg), frame='icrs')
    galex = Table({key:np.full_like(source_table['idx'],np.nan,float) for key in list(galex_zp_table['col'])+list(galex_zp_table['err_col'])})
    galex_dr7 = Vizier.query_region(coords, radius=2*u.arcsec,catalog=['II/335/galex_ais'])[0]
    
    galex['idx'] = source_table['idx']
    galex_dr7['NUVmag'].name = 'nuv_mag'
    galex_dr7['e_NUVmag'].name = 'nuv_magerr'
    galex_dr7['FUVmag'].name = 'fuv_mag'
    galex_dr7['e_FUVmag'].name = 'fuv_magerr'

    galex_dr7['_q'] = galex_dr7['_q'] - 1
    galex_dr7['_q'] = source_table['idx'][galex_dr7['_q']]
    galex_dr7['_q'].name = 'idx'
    for i in range(len(coords)):
        data = Catalogs.query_region(coords[i], radius=2*u.arcsec,catalog='Galex')
        if len(data) == 0: ## no GALEX data found for this source
            continue 
        for col in galex_zp_table['col']:
            if np.ma.is_masked(data[col][0]): ## GALEX data is missing in this band 
                continue
            err_col = galex_zp_table[galex_zp_table['col']==col]['err_col'][0]
            f0 = galex_zp_table[galex_zp_table['col']==col]['f0'][0]
            m0 = galex_zp_table[galex_zp_table['col']==col]['m0'][0]
            galex[i][col] = f0 * 10**(-0.4*(data[col][0]-m0))
            galex[i][err_col] = galex[i][col] * 0.4 * np.log(10) * data[err_col][0]
            if galex[i]['idx'] in galex_dr7['idx']: ## if there is a match in the GALEX DR7 catalog, use that data instead
                j = np.where(galex_dr7['idx']==galex[i]['idx'])[0][0]
                if not np.ma.is_masked(galex_dr7[j][col]):
                    galex[i][col] = f0 * 10**(-0.4*(galex_dr7[j][col]-m0))
                    galex[i][err_col] = galex[i][col] * 0.4 * np.log(10) * galex_dr7[j][err_col]
            
    for col in galex_zp_table['col']:
        band = galex_zp_table[galex_zp_table['col']==col]['band'][0]
        err_col = galex_zp_table[galex_zp_table['col']==col]['err_col'][0]
        galex[col].unit = u.erg / u.s / u.cm**2 / u.AA
        galex[col].name = band
        galex[err_col].unit = u.erg / u.s / u.cm**2 / u.AA
        galex[err_col].name = band + '_err'
    return galex
        
def get_photometry(source_table):
    if not ('idx' in source_table.colnames):
        source_table['idx'] = np.arange(len(source_table))
    tbl = source_table['idx','source_id','ra','dec','parallax','parallax_error','[Fe/H]','Av']
    tbl = table.join(tbl, get_gaia_photometry(source_table), keys='source_id', join_type='left',metadata_conflicts='silent')
    tbl = table.join(tbl, get_synthetic_photometry(tbl), keys='source_id', join_type='left',metadata_conflicts='silent')
    tbl = table.join(tbl, get_galex_photometry(source_table), keys='idx', join_type='left',metadata_conflicts='silent')
    tbl = table.join(tbl, get_twomass_photometry(source_table), keys='idx', join_type='left',metadata_conflicts='silent')
    tbl = table.join(tbl, get_wise_photometry(source_table), keys='idx', join_type='left',metadata_conflicts='silent')

    flux_cols = list(gaia_zp_table['band']) + list(wise_zp_table['band']) + list(twomass_zp_table['band']) + list(synt_zp_table['band']) + list(galex_zp_table['band'])
    for col in flux_cols:
        snr = tbl[col] / tbl[col + '_err']
        for i in range(len(tbl)): 
            if snr[i] > 10: ## if SNR > 10, set error to 10% of flux: minimal error to account for model uncertainties
                tbl[i][col+'_err'] = 0.1 * tbl[i][col]

    Table.sort(tbl,'idx')
    return tbl

In [4]:
## model and fitting functions

filter_list = ['Obs.Flux_2MASS.H', 'Obs.Flux_2MASS.J', 'Obs.Flux_2MASS.Ks',
                'Obs.Flux_ACS_WFC.F606W', 'Obs.Flux_ACS_WFC.F814W',
                  'Obs.Flux_APASS.B', 'Obs.Flux_APASS.V',
                    'Obs.Flux_DECam.Y', 'Obs.Flux_DECam.g', 'Obs.Flux_DECam.i', 'Obs.Flux_DECam.r', 'Obs.Flux_DECam.z',
                      'Obs.Flux_DENIS.I', 'Obs.Flux_DENIS.J', 'Obs.Flux_DENIS.Ks',
                        'Obs.Flux_GAIA3.G', 'Obs.Flux_GAIA3.Gbp', 'Obs.Flux_GAIA3.Grp', 'Obs.Flux_GAIA3.Grvs',
                          'Obs.Flux_GALEX.FUV', 'Obs.Flux_GALEX.NUV',
                            'Obs.Flux_Johnson.B', 'Obs.Flux_Johnson.I', 'Obs.Flux_Johnson.R', 'Obs.Flux_Johnson.U', 'Obs.Flux_Johnson.V',
                              'Obs.Flux_PS1.g', 'Obs.Flux_PS1.i', 'Obs.Flux_PS1.r', 'Obs.Flux_PS1.y', 'Obs.Flux_PS1.z',
                                'Obs.Flux_SDSS.g', 'Obs.Flux_SDSS.i', 'Obs.Flux_SDSS.r', 'Obs.Flux_SDSS.u', 'Obs.Flux_SDSS.z',
                                  'Obs.Flux_UKIDSS.K',
                                    'Obs.Flux_VISTA.H', 'Obs.Flux_VISTA.J', 'Obs.Flux_VISTA.Ks', 'Obs.Flux_VISTA.Y',
                                      'Obs.Flux_WISE.W1', 'Obs.Flux_WISE.W2', 'Obs.Flux_WISE.W3']

def get_distance(parallax,parallax_err):
    ## distance in meters calculated from parallax in mas
    return (1000/parallax) * pc.value , (1000/parallax**2)*parallax_err * pc.value

def blackbody_flux(wavelength, temperature):
    ## wavelength and temperature must have astropy units
    f = (2 * np.pi * h.cgs * c.cgs**2 / wavelength.cgs**5) / (np.exp(h.cgs * c.cgs / (wavelength.cgs * k_B.cgs * temperature)) - 1)
    return f.to(u.erg / u.s / u.cm**2 / u.AA)

def blackbody_mod_table(teff,radius,parallax,parallax_err,Av):
    bands = list(bands_table['band'])
    mod_tbl = Table(dict(zip(bands,[[float(1)] for i in range(len(bands))])))
    d, d_err = get_distance(parallax,parallax_err)
    radius = radius * R_sun.value

    AW1,AW2,AW3,AW4 = extinction.get_WISE_extinction(Av,0,teff)
    AJ,AH,AKs = extinction.get_2MASS_extinction(Av,0,teff)
    AU,AB,AV,AR,AI = extinction.get_Johnson_extinction(Av,0,teff)
    Au,Ag,Ar,Ai,Az = extinction.get_SDSS_extinction(Av,0,teff)
    AG,AGbp,AGrp = extinction.get_Gaia_extinction(Av,0,teff)
    AFUV,ANUV = extinction.get_Galex_extinction(Av,0,teff)

    ext_dict = {'2MASS.J':AJ,'2MASS.H':AH,'2MASS.Ks':AKs,'GALEX.FUV':AFUV,'GALEX.NUV':ANUV,'GAIA3.G':AG,'GAIA3.Gbp':AGbp,'GAIA3.Grp':AGrp,
                'WISE.W1':AW1,'WISE.W2':AW2,'WISE.W3':AW3,'WISE.W4':AW4,'Johnson.U':AU,'Johnson.B':AB,'Johnson.V':AV,'Johnson.R':AR,'Johnson.I':AI,
                'SDSS.u':Au,'SDSS.g':Ag,'SDSS.r':Ar,'SDSS.i':Ai,'SDSS.z':Az}

    for b in bands:
        filepath = os.path.join('..','data','VOSA','filters',b+'.dat')
        filter_tbl = Table.read(filepath, format='ascii', names=['wavelength','transmission'])
        wavelength = filter_tbl['wavelength'].data * u.AA
        
        transmission = filter_tbl['transmission'].data 
        flux = blackbody_flux(wavelength, teff * u.K)
        flux = np.dot(flux, transmission)
        flux /= np.sum(transmission)
        flux = flux * (radius/d)**2
        mod_tbl[0][b] = flux.value
        mod_tbl[b].unit = flux.unit 
        mod_tbl[b] = mod_tbl[b] * 10**(-0.4 * ext_dict[b])

    return mod_tbl
        
def get_chi2(obs_tbl_dered, mod_tbl, no_uv = False):
    chi2 = 0
    ndof = 0
    for col in mod_tbl.colnames:
        if no_uv and col.startswith('GALEX'):
           continue
        if col in obs_tbl_dered.colnames:
            if not(np.ma.is_masked(obs_tbl_dered[col]) or np.ma.is_masked(obs_tbl_dered[col + '_err'])):
                chi2 += np.sum((obs_tbl_dered[col] - mod_tbl[col])**2 /(0.1 * obs_tbl_dered[col])**2)
                ndof += 1
    return chi2/ndof

def plot_obs_sed(sources,idx,lam_f_lam=False,model=None,title='SED',no_uv = True, no_IR = False):
    warnings.simplefilter('ignore', UserWarning)
    obs_tbl = get_photometry(sources)
    # obs_tbl_dered = deredden_obs_table(obs_tbl)

    bnds = list(bands_table['band'])
    wl = np.array(list(bands_table['lambda_eff']))
    flux = np.array([obs_tbl[idx][bnd] for bnd in bnds])
    flux_err = 0.1 * flux

    if model is not None:
        # fig, (ax,ax2) = plt.subplots(2,1,sharex=True,gridspec_kw={'height_ratios': [3, 1], 'hspace':0},figsize=(1.5*figwidth_single, figwidth_single), tight_layout=True,dpi=300)
        fig, ax = plt.subplots(figsize=(1.5*figwidth_single, figwidth_single), tight_layout=True,dpi=300)
        teff, radius, success, chi2 = model
        mod_tbl = blackbody_mod_table(teff,radius,obs_tbl[idx]['parallax'],obs_tbl[idx]['parallax_error'],obs_tbl[idx]['Av'])
        mod_flux = np.array([mod_tbl[bnd][0] for bnd in bnds])
      
        if lam_f_lam:
          mod_flux = wl * mod_flux
          ax.plot(wl, mod_flux,color='RoyalBlue',ls='--',label=f'BB {int(teff)}K/{radius:.2f} $R_\odot$',marker='.',mec='w',mfc='w',markersize=3, zorder = 0)
          ax.set_ylabel(r'$\lambda f_\lambda $($erg/s/cm^2/\AA$)',fontsize=8)
        else:
          ax.plot(wl,mod_flux,color='RoyalBlue',ls='--',label=f'BB {int(teff)}K/{radius:.2f} $R_\odot$',marker='.',mec='w',mfc='w',markersize=3, zorder = 0)
          ax.set_ylabel(r'$f_\lambda $($erg/s/cm^2/\AA$)',fontsize=8)
        # ax2.scatter(wl,(flux - mod_flux) / (flux_err),marker='.',c='Crimson')
        # ax2.hlines(0,wl[0],wl[-1],color='k',ls='--')
        # ax2.grid()
        # ax2.set_ylabel('Residuals/error',fontsize=8,labelpad=10)
        # ax2.tick_params(axis='both', which='major', labelsize=8)
        # ax2.set_xlabel(r'wavelength $(\AA)$',fontsize=8)
        textstr = 'Fit success: ' + str(bool(success)) + f'\n $\chi^2/ndof$: {chi2:.2f}'
        props = dict(boxstyle='round', facecolor='Silver', alpha=0.9)
        ax.text(0.08, 0.2, textstr, transform=ax.transAxes, fontsize=8,
            verticalalignment='top', bbox=props, color='k')
    else:
       fig,ax = plt.subplots(figsize=(figwidth_single, figwidth_single),dpi=300)
       ax.set_xlabel(r'wavelength $(\AA)$',fontsize=8)
    if lam_f_lam:
      flux = wl * flux
      ax.errorbar(wl,flux,yerr = wl * flux_err,fmt='.',label='observed',ecolor='k',elinewidth=2,mec='Crimson',mfc='Crimson',capsize=2,markersize=1, zorder = 5)
      ax.set_ylabel(r'$\lambda f_\lambda $($erg/s/cm^2/\AA$)',fontsize=8)
    else:
       ax.errorbar(wl,flux,yerr = flux_err,fmt='.',label='observed',ecolor='Salmon',elinewidth=2,mec='Crimson',mfc='Crimson',capsize=2,markersize=1, zorder = 5)
       ax.set_ylabel(r'$f_\lambda $($erg/s/cm^2/\AA$)',fontsize=8)
    if no_uv:
        ax.scatter(wl[wl<2500],flux[wl<2500],marker='.',c='Gold',label='No fit',s=3,zorder = 10)
    if no_IR:
        ax.scatter(wl[wl>10000],flux[wl>10000],marker='.',c='Gold',s=3,zorder = 10)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_title(title,fontsize=8)
    ax.set_facecolor('0.25')
    ax.tick_params(axis='both', which='major', labelsize=8)
    ax.grid()
    ax.legend(fontsize=8,loc='best')
    fig.show()
    return fig

def fit_table(sources, init = (6000,1) ,teff_bounds = (3000,10000), r_bounds = (0.4,3.0), no_uv = True, no_IR = []):
    ## init, teff_bounds, r_bounds either a tuple or a list of tuples of length len(sources)
    ## if list of tuples, each tuple is for a different source
    obs_tbl = get_photometry(sources)
    # obs_tbl_dered = deredden_obs_table(obs_tbl)
    results = Table(np.zeros((len(sources),6)), names= ['teff1','teff1_err','r1','r1_err','chi2/ndof','no_IR'])
    
    if type(init) == tuple:
      init = [init for i in range(len(sources))]
    elif len(init) != len(sources):
      raise ValueError('init must be a tuple or a list of tuples of length len(sources)')
    if type(teff_bounds) == tuple:
      teff_bounds = [teff_bounds for i in range(len(sources))]
    elif len(teff_bounds) != len(sources):
      raise ValueError('teff_bounds must be a tuple or a list of tuples of length len(sources)')
    if type(r_bounds) == tuple:
      r_bounds = [r_bounds for i in range(len(sources))]
    elif len(r_bounds) != len(sources):
      raise ValueError('r_bounds must be a tuple or a list of tuples of length len(sources)')
    
    i = 0
    for obs_row,init,teff_bounds,r_bounds in tqdm(zip(obs_tbl,init,teff_bounds,r_bounds),total=len(sources)):
      idx = sources['idx'][i]
      bnds = list(bands_table['band'])
      if no_uv:
        bnds = [b for b in bnds if not b.startswith('GALEX')]
      if idx in no_IR:
         bnds = [b for b in bnds if not (b.startswith('2MASS') or b.startswith('WISE'))]
      bnds = [b for b in bnds if not (np.ma.is_masked(obs_row[b]) or np.isnan(obs_row[b]))] 
      wl = np.array([bands_table[bands_table['band']==bnd]['lambda_eff'][0] for bnd in bnds])
      
      def bb_fun(wl,teff,radius):
        mod_tbl = blackbody_mod_table(teff,radius,obs_row['parallax'],obs_row['parallax_error'],obs_row['Av'])
        mod_flux = np.array([mod_tbl[bnd][0] for bnd in bnds])
        return mod_flux

      flux = np.array([obs_row[bnd] for bnd in bnds])
      flux_err = 0.1 * flux
      vals, covm = curve_fit(bb_fun, wl, flux, p0 = list(init), bounds = ([teff_bounds[0],r_bounds[0]],[teff_bounds[1],r_bounds[1]]), sigma = flux_err, absolute_sigma = True)
      results[i]['teff1'] = vals[0]
      results[i]['teff1_err'] = covm[0,0]**(1/2)
      results[i]['r1'] = vals[1]
      results[i]['r1_err'] = covm[1,1]**(1/2)
      results[i]['chi2/ndof'] = np.sum((flux - bb_fun(wl,vals[0],vals[1]))**2 / flux_err**2) / (len(flux) - len(vals))
      results[i]['no_IR'] = idx in no_IR
      i += 1
    return results

In [7]:
## fitting the table

sources = Table.read('../table_C.fits', format='fits')
init_lst = [(6000,sources[i]['m1']) for i in range(len(sources))]
r_bounds_lst = [(0.3*sources[i]['m1'],3*sources[i]['m1']) for i in range(len(sources))]
teff_bounds_lst = [(3000,20000) for i in range(len(sources))]
no_IR = []
results = fit_table(sources, init = init_lst ,teff_bounds = teff_bounds_lst, r_bounds = r_bounds_lst, no_uv = True , no_IR = no_IR)


# saving the results

sources['teff1'] = results['teff1']
sources['teff1_err'] = results['teff1_err']
sources['r1'] = results['r1']
sources['r1_err'] = results['r1_err']
sources['chi2/ndof'] = results['chi2/ndof']
sources['no_IR'] = results['no_IR']

fail = []
sources['fit_success'] = [False if np.isin(i,fail) else True for i in range(len(sources))]
# sources.write('../table_C.fits', format='fits', overwrite=True)

# plotting the results

for j in range(len(sources)):
    fig = plot_obs_sed(sources,j,lam_f_lam=True,model=sources[j]['teff1','r1','fit_success','chi2/ndof'],title=f'Candidate {sources[j]["idx"]} SED fitting', no_uv = True, no_IR = sources[j]['no_IR'])
    fig.savefig(f'../img/bb_fits/bb_{sources[j]["idx"]}.png',bbox_inches='tight')
    plt.close(fig)

100%|██████████| 35/35 [04:04<00:00,  6.98s/it]


# Summing different models

In [21]:
model = WD_models.load_model(low_mass_model='Bedard2020',
                             middle_mass_model='Bedard2020',
                             high_mass_model='ONe',
                             atm_type='H',
                             HR_bands=('FUV-NUV', 'NUV'))

sources = Table.read('../table_C.fits',format='fits')

m_agecool_to_logteff = WD_models.interp_xy_z_func(x=model['mass_array'],
                                                  y=model['age_cool'],
                                                  z=model['logteff'],
                                                  interp_type='linear')

m_agecool_to_mag = WD_models.interp_xy_z_func(x=model['mass_array'],
                                                y=model['age_cool'],
                                                z=model['Mag'],
                                                interp_type='linear')

teff2 = []
for m2,age in zip(sources['m2'],sources['age']):
    teff2.append(10**m_agecool_to_logteff(m2,age * 1e-3))
sources['teff2_min'] = teff2

In [6]:
## model SED functions

ms_sed_logg_arr = np.arange(0,5.1,0.5)
wd_sed_logg_arr = np.arange(6.5,9.7,0.25)
meta_sed_arr = np.array([-2.5,2.0,-1.5,-1.0,-0.5,0.0,0.2,0.5])
ms_sed_teff_arr = np.hstack([np.arange(3500,13000,250),np.arange(13000,50001,1000)])
wd_sed_teff_arr = np.hstack([np.arange(5000,20000,250),np.arange(20000,30000,1000),np.arange(30000,40000,2000),np.arange(40000,50000,5000),np.arange(50000,80001,10000)])


filter_band_dict = {'GALEX/GALEX.FUV':'GALEX.FUV','GALEX/GALEX.NUV':'GALEX.NUV','GAIA/GAIA3.G':'GAIA3.G','GAIA/GAIA3.Gbp':'GAIA3.Gbp','GAIA/GAIA3.Grp':'GAIA3.Grp',
    'Generic/Johnson.U':'Johnson.U','Generic/Johnson.B':'Johnson.B','Generic/Johnson.V':'Johnson.V','Generic/Johnson.R':'Johnson.R','Generic/Johnson.I':'Johnson.I'
        ,'SLOAN/SDSS.u':'SDSS.u','SLOAN/SDSS.g':'SDSS.g','SLOAN/SDSS.r':'SDSS.r','SLOAN/SDSS.i':'SDSS.i','SLOAN/SDSS.z':'SDSS.z',
        '2MASS/2MASS.J':'2MASS.J','2MASS/2MASS.H':'2MASS.H','2MASS/2MASS.Ks':'2MASS.Ks',
        'WISE/WISE.W1':'WISE.W1','WISE/WISE.W2':'WISE.W2','WISE/WISE.W3':'WISE.W3','WISE/WISE.W4':'WISE.W4'}


def get_kurucz_sed_filepath(teff, logg, alpha, meta):
    ## 3500 <= teff <= 13000 with jumps of 250 (additional templates up to 50,000, but less dense)
    ## 0.0 <= logg <= 5.0 with jumps of 0.5
    ## alpha = 0 or alpha = 0.4
    ## -4.0 <= metallicity <= 0.5 evaluated at: -4.0, -2.5, -2.0, -1.5, -1.0, -0.5, 0.0, 0.2, 0.5

    logg = ms_sed_logg_arr[np.argmin(np.abs(ms_sed_logg_arr - logg))]
    teff = ms_sed_teff_arr[np.argmin(np.abs(ms_sed_teff_arr - teff))]
    meta = meta_sed_arr[np.argmin(np.abs(meta_sed_arr - meta))]

    dirname = os.path.join('..','data','VOSA','models_MS','Kurucz')
    filename = 'Kurucz2003all_f'
    if meta >= 0:
        filename += 'p'
    elif meta <= 0:
        filename += 'm'
    meta_dig1 = int((abs(meta)*10)//10)
    meta_dig2 = int((abs(meta)*10)%10)
    filename += f'{meta_dig1}{meta_dig2}'
    if alpha > 0:
        filename += 'a'
    filename += f't{int(teff)}'
    logg_dig1 = int((logg*10)//10)
    logg_dig2 = int((logg*10)%10)
    filename += f'g{logg_dig1}{logg_dig2}'
    filename += 'k2odfnew.fl.phot.xml'
    path = os.path.join(dirname,filename)
    if os.path.exists(path):
        return path
    elif os.path.exists(path.replace('all','')):
        return path.replace('all','')
    else:
        print('no such template ' + filename)
        return ''
    
def get_koester_sed_filepath(teff, logg):
    ## 5,000 <= teff <= 50,000 with jumps of 250
    ## 6.5 <= logg <= 9.0 with jumps of 0.25

    logg = wd_sed_logg_arr[np.argmin(np.abs(wd_sed_logg_arr - logg))]
    teff = wd_sed_teff_arr[np.argmin(np.abs(wd_sed_teff_arr - teff))]
    
    dirname = os.path.join('..','data','VOSA','models_WD','Koester')
    filename = 'koester2_da'
    teff_dig1 = int((teff//10000)%10)
    teff_dig2 = int((teff//1000)%10)
    teff_dig3 = int((teff//100)%10)
    teff_dig4 = int((teff//10)%10)
    teff_dig5 = int((teff//1)%10)
    filename += f'{teff_dig1}{teff_dig2}{teff_dig3}{teff_dig4}{teff_dig5}_'
    filename += f'{int(logg*100)}'
    filename += '.dk.phot.xml'
    path = os.path.join(dirname,filename)
    if os.path.exists(path):
        return path
    else:
        # print('no such template ' + filename)
        return ''
    
def get_radius(logg, m, m_err):
    ## calculate radius (in meters) from logg and the stellar mass (in units of M_sun)
    ## note to self- log g is defined as log_10 of g in cm/s^2
    r = np.sqrt(G.value * m * M_sun.value / (10**logg / 100))
    dr = r / (2*m) * m_err 
    return r , dr

def inverse_square_coeff(logg,m,m_err,parallax,parallax_err):
    r,dr = get_radius(logg,m,m_err)
    d,d_err = get_distance(parallax,parallax_err)
    c = (r/d)**2
    dc = c * np.sqrt((2*dr/r)**2 + (2*d_err/d)**2)
    return c,dc

def get_ms_logg(m, m_err):
    ## note to self- log g is defined as log_10 of g in cm/s^2

    ## use main sequence scaling relation: mass proportional to radius
    r = m * R_sun
    dr = m_err * R_sun
    g = G.value * m * M_sun.value / r**2 * 1e2 ## in cm/s^2
    dg = g * np.sqrt((2*dr/r)**2 + (2*m_err/m)**2)
    return np.log10(g.value)

def get_wd_logg(m,teff):
    ## WD radius nearly independent of mass
    m_logteff_to_logg = WD_models.interp_xy_z_func(x=model['mass_array'],
                                                  y=model['logteff'],
                                                  z=model['logg'],
                                                  interp_type='linear')
    logg = float(m_logteff_to_logg(m,np.log10(teff)))
    return logg

def model_sed_ms(teff1, m1, m1_err, parallax, parallax_err, meta1 = -0.5, alpha1 = 0, r1 = 0):
    ## teff and logg are the parameters we wanna optimize (maybe alpha and meta too, one day)
    ## m (mass) and parallax should be given, to be used in the flux calculation
    if r1 == 0:
        logg1 = get_ms_logg(m1,m1_err)
    else:
        logg1 = np.log10(G.value * m1 * M_sun.value / (r1 * R_sun.value)**2 * 1e2)
    
    teff_idx = np.searchsorted(ms_sed_teff_arr,teff1)

    if teff_idx == 0 or teff_idx == len(ms_sed_teff_arr):
        ## take the closest template
        path1 = get_kurucz_sed_filepath(teff1, logg1, alpha1, meta1)
        if len(path1) == 0: ## wrong path: change directory path / check the parameters indeed have a template
            return Table({'filter':[], 'wavelength':[],'flux':[],'flux_err':[]}) 
        c1,dc1 = inverse_square_coeff(logg1,m1,m1_err,parallax,parallax_err)
        votable1 = parse(path1)
        table1 = Table.read(votable1.get_first_table())
        model_flux = c1 * table1['flux'].data * table1['wavelength'].data
        model_flux_err = dc1 * table1['flux'].data
        model_table = Table({'filter':table1['filter'].data, 'wavelength':table1['wavelength'].data,'flux':model_flux,'flux_err':model_flux_err})
        model_table.sort('wavelength')
    else:
        ## interpolate between two closest templates
        teff_r = ms_sed_teff_arr[teff_idx]
        teff_l = ms_sed_teff_arr[teff_idx-1]
        pathr = get_kurucz_sed_filepath(teff_r, logg1, alpha1, meta1)
        pathl = get_kurucz_sed_filepath(teff_l, logg1, alpha1, meta1)
        if len(pathr) == 0: ## wrong path: change directory path / check the parameters indeed have a template
            return Table({'filter':[], 'wavelength':[],'flux':[],'flux_err':[]}) 
        c1,dc1 = inverse_square_coeff(logg1,m1,m1_err,parallax,parallax_err)
        vtr = parse(pathr)
        vtl = parse(pathl)
        tabler = Table.read(vtr.get_first_table())
        tablel = Table.read(vtl.get_first_table())
        model_flux = c1 *(tabler['flux'].data * (teff1 - teff_l) + tablel['flux'].data * (teff_r - teff1))* tablel['wavelength'].data / (teff_r - teff_l)
        model_flux_err = dc1 / c1 * model_flux
        model_table = Table({'filter':tabler['filter'].data, 'wavelength':tabler['wavelength'].data,'flux':model_flux,'flux_err':model_flux_err})
        model_table.sort('wavelength')
    return model_table

def model_sed_wd(teff2,m2,m2_err,parallax,parallax_err):
      
    logg2 = get_wd_logg(m2,teff2)
    teff_idx = np.searchsorted(wd_sed_teff_arr,teff2)

    if teff_idx == 0 or teff_idx == len(wd_sed_teff_arr):
        ## take the closest template
        path2 = get_koester_sed_filepath(teff2, logg2)
        if len(path2) == 0: ## wrong path: change directory path / check the parameters indeed have a template
            return Table({'filter':[], 'wavelength':[],'flux':[],'flux_err':[]}) 
        c2,dc2 = inverse_square_coeff(logg2,m2,m2_err,parallax,parallax_err)
        votable2 = parse(path2)
        table2 = Table.read(votable2.get_first_table())
        model_flux = c2 * table2['flux'].data * table2['wavelength'].data
        model_flux_err = dc2 * table2['flux'].data
        model_table = Table({'filter':table2['filter'].data, 'wavelength':table2['wavelength'].data,'flux':model_flux,'flux_err':model_flux_err})
        model_table.sort('wavelength')
    else:
        ## interpolate between two closest templates
        teff_r = wd_sed_teff_arr[teff_idx]
        teff_l = wd_sed_teff_arr[teff_idx-1]
        pathr = get_koester_sed_filepath(teff_r, logg2)
        pathl = get_koester_sed_filepath(teff_l, logg2)
        if len(pathr) == 0: ## wrong path: change directory path / check the parameters indeed have a template
            return Table({'filter':[], 'wavelength':[],'flux':[],'flux_err':[]})
        c2,dc2 = inverse_square_coeff(logg2,m2,m2_err,parallax,parallax_err)
        vtr = parse(pathr)
        vtl = parse(pathl)
        tabler = Table.read(vtr.get_first_table())
        tablel = Table.read(vtl.get_first_table())
        if np.isin('WISE/WISE.W4',tabler['filter'].data):
            tabler.remove_row(np.where(tabler['filter'].data == 'WISE/WISE.W4')[0][0])
        if np.isin('WISE/WISE.W4',tablel['filter'].data):
            tablel.remove_row(np.where(tablel['filter'].data == 'WISE/WISE.W4')[0][0])
        model_flux = c2 *(tabler['flux'].data * (teff2 - teff_l) + tablel['flux'].data * (teff_r - teff2))* tablel['wavelength'].data / (teff_r - teff_l)
        model_flux_err = dc2 / c2 * model_flux
        model_table = Table({'filter':tabler['filter'].data, 'wavelength':tabler['wavelength'].data,'flux':model_flux,'flux_err':model_flux_err})
        model_table.sort('wavelength')
    
    return model_table

def plot_sed_ms_wd(teff1, m1, m1_err, teff2, m2, m2_err, parallax, parallax_err, meta1 = 0):
    ms = model_sed_ms(teff1, m1, m1_err, parallax, parallax_err, meta1)
    wd = model_sed_wd(teff2, m2, m2_err, parallax, parallax_err)

    wl = np.intersect1d(ms['wavelength'].data,wd['wavelength'].data)
    ms = ms[np.isin(ms['wavelength'],wl)]
    wd = wd[np.isin(wd['wavelength'],wl)]

    tot_flux = ms['flux'] + wd['flux']

    fig , ax = plt.subplots(figsize=(figwidth_single, figwidth_single), tight_layout=True,dpi=300)
    ax.plot(ms['wavelength'],ms['flux'],linestyle='--',color='RoyalBlue',label=f'MS {int(teff1)}K',linewidth=1,alpha=0.8)
    ax.plot(wd['wavelength'],wd['flux'],linestyle='--',color='Crimson',label=f'WD {int(teff2)}K',linewidth=1,alpha=0.8)
    ax.plot(ms['wavelength'],tot_flux,color='k',label=f'total flux',linewidth=1,marker='.',markersize=5)
    ax.scatter(ms['wavelength'][1],tot_flux[1],color='Indigo',s=50,label='Galex NUV',marker='o')
    ax.legend(loc = 'best')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel(r'$\lambda (\AA) $',fontdict={'size':12})
    ax.set_ylabel(r'$\lambda F_\lambda(erg/s/cm^2)$',fontdict={'size':12})
    ax.set_facecolor('0.7')
    ax.set_ylim(1e-20,10*np.max(ms['flux']))
    ax.legend()
    ax.grid()
    fig.show()

def plot_kurucz_vs_obs(sources,idx,plot=False,save=False):
    warnings.simplefilter('ignore', UserWarning)
    obs_tbl = get_photometry(sources)
    # obs_tbl_dered = deredden_obs_table(obs_tbl)
    i = np.where(sources['idx'] == idx)[0][0]
    
    bnds = list(bands_table['band'])
    wl = np.array(list(bands_table['lambda_eff']))
    flux = np.array([obs_tbl[i][bnd] for bnd in bnds])
    flux = flux * wl
    flux_err = 0.1 * flux

    teff1 = sources[i]['teff1']
    teff1_err = sources[i]['teff1_err']
    teff_low = teff1 + teff1_err 
    teff_hi = teff1 - teff1_err
    r1 = sources[i]['r1']
    r1_err = sources[i]['r1_err']
    r_lo = r1 - r1_err
    r_hi = r1 + r1_err
    m = sources[i]['m1']
    m_err = sources[i]['m1_err']
    parallax = sources[i]['parallax']
    parallax_err = sources[i]['parallax_error']
    meta = sources[i]['[Fe/H]']
    av = sources[i]['Av']
    ms_lo = model_sed_ms(teff_low, m - m_err, m_err, parallax, parallax_err, meta1 = meta, r1 = r_lo)
    ms_hi = model_sed_ms(teff_hi, m + m_err, m_err, parallax, parallax_err, meta1 = meta, r1 = r_hi)
    ms_lo = redden_model_table(ms_lo,teff_low,av)
    ms_hi = redden_model_table(ms_hi,teff_hi,av)

    fig, ax = plt.subplots(figsize=(figwidth_single, 0.8*figwidth_single), tight_layout=True, dpi=300)

    ax.fill_between(ms_lo['wavelength'],ms_lo['flux'],ms_hi['flux'],color='RoyalBlue',alpha=0.5,label=f'Best fit Kurucz model + errors')
   
    ax.set_ylabel(r'$\lambda f_\lambda $($erg/s/cm^2/\AA$)',fontsize=8)
    ax.set_xlabel(r'wavelength $(\AA)$',fontsize=8)
    ax.errorbar(wl, flux,yerr = flux_err,fmt='.',label='observed',ecolor='k',elinewidth=2,mec='Crimson',mfc='Crimson',capsize=2,markersize=1)

    if sources[i]['no_IR'] and len(flux[wl>10000]) > 0:
        ax.scatter(wl[wl>10000],flux[wl>10000],marker='.',color='Gold',s=1,zorder = 10)
    ax.scatter(wl[wl<3500],flux[wl<3500],marker='.',color='Gold',s=1,zorder = 10)

    ax.set_ylabel(r'$\lambda f_\lambda $($erg/s/cm^2/\AA$)',fontsize=8)

    from matplotlib.ticker import FormatStrFormatter
    ax.set_xticks([2e3,5e3,8e3,1e4,2e4,5e4,1e5])
    ax.get_xaxis().set_major_formatter(FormatStrFormatter('%.0e'))
    ax.set_xlim(1e3,1.2*wl[~np.isnan(flux)].max())
    ax.set_xscale('log')
    ax.tick_params(axis='x', which='major', labelsize=7)

    ax.set_yscale('log')
    ax.tick_params(axis='y', which='major', labelsize=8)
    
    ax.legend(fontsize=8,loc='lower right',frameon=False)
    ax.set_title(f'candidate {idx}',fontsize=7)
    fig.show()
    if save:
        fig.savefig(f'../img/kurucz/kurucz_{idx}.png')
    if not plot:
        plt.close(fig)

def redden_model_table(mod_tbl,teff,av):
    AW1,AW2,AW3,AW4 = extinction.get_WISE_extinction(av,0,teff)
    AJ,AH,AKs = extinction.get_2MASS_extinction(av,0,teff)
    AU,AB,AV,AR,AI = extinction.get_Johnson_extinction(av,0,teff)
    Au,Ag,Ar,Ai,Az = extinction.get_SDSS_extinction(av,0,teff)
    AG,AGbp,AGrp = extinction.get_Gaia_extinction(av,0,teff)
    AFUV,ANUV = extinction.get_Galex_extinction(av,0,teff)
    ext_dict = {'2MASS.J':AJ,'2MASS.H':AH,'2MASS.Ks':AKs,'GALEX.FUV':AFUV,'GALEX.NUV':ANUV,'GAIA3.G':AG,'GAIA3.Gbp':AGbp,'GAIA3.Grp':AGrp,
                'WISE.W1':AW1,'WISE.W2':AW2,'WISE.W3':AW3,'WISE.W4':AW4,'Johnson.U':AU,'Johnson.B':AB,'Johnson.V':AV,'Johnson.R':AR,'Johnson.I':AI,
                'SDSS.u':Au,'SDSS.g':Ag,'SDSS.r':Ar,'SDSS.i':Ai,'SDSS.z':Az}
    for fil in mod_tbl['filter']:
        i = np.where(mod_tbl['filter'] == fil)[0][0]
        if fil in filter_band_dict.keys():
            band = filter_band_dict[fil]
            mod_tbl['flux'][i] = mod_tbl['flux'][i] * 10**(-0.4 * ext_dict[band])
        else:
            mod_tbl.remove_row(i)
    return mod_tbl
    

In [16]:
## model spectrum functions

ms_logg_arr = np.array([3.5,4.0,4.5,5.0])
wd_logg_arr = np.arange(6.5,9.7,0.25)
meta_arr = np.array([-2.5,2.0,-1.5,-1.0,-0.5,0.0,0.2,0.5])
ms_teff_arr = np.hstack([np.arange(3500,13000,250),np.arange(13000,50001,1000)])
wd_teff_arr = np.hstack([np.arange(5000,20000,250),np.arange(20000,30000,1000),np.arange(30000,40000,2000),np.arange(40000,50000,5000),np.arange(50000,80000,10000)])

def get_kurucz_spec_filepath(teff, logg, meta):
    ## 3500 <= teff <= 50000 with jumps of 250 up to 13,000
    ## 3.5 <= logg <= 5.0 with jumps of 0.5
    ## -2.5 <= metallicity <= 0.5 evaluated at: -2.5, -2.0, -1.5, -1.0, -0.5, 0.0, 0.2, 0.5
    dirname = os.path.join('..','data','VOSA','spectra_MS','Kurucz')
    filename = 'f'

    logg = ms_logg_arr[np.argmin(np.abs(ms_logg_arr - logg))]
    meta = meta_arr[np.argmin(np.abs(meta_arr - meta))]
    teff = ms_teff_arr[np.argmin(np.abs(ms_teff_arr - teff))]

    if meta >= 0:
        filename += 'p'
    elif meta <= 0:
        filename += 'm'
    meta_dig1 = int((abs(meta)*10)//10)
    meta_dig2 = int((abs(meta)*10)%10)
    filename += f'{meta_dig1}{meta_dig2}'
    filename += 'k2odfnew.pck.teff='
    filename += f'{int(teff)}..logg='
    filename += f'{logg:0.5f}'
    filename += '.dat.xml'
    path = os.path.join(dirname,filename)
    if os.path.exists(path):
        return path
    else:
        print('no such template ' + filename)
        return ''
    
def get_koester_spec_filepath(teff, logg):
    ## 5,000 <= teff <= 80,000 with jumps of 250
    ## 7.0 <= logg <= 8.0 with jumps of 0.25
    dirname = os.path.join('..','data','VOSA','spectra_WD','Koester')
    filename = 'da'
    
    logg = wd_logg_arr[np.argmin(np.abs(wd_logg_arr - logg))]
    teff = wd_teff_arr[np.argmin(np.abs(wd_teff_arr - teff))]

    teff_dig1 = int((teff//10000)%10)
    teff_dig2 = int((teff//1000)%10)
    teff_dig3 = int((teff//100)%10)
    teff_dig4 = int((teff//10)%10)
    teff_dig5 = int((teff//1)%10)
    filename += f'{teff_dig1}{teff_dig2}{teff_dig3}{teff_dig4}{teff_dig5}_'
    filename += f'{int(logg*100)}'
    filename += '.dk.dat.xml'
    path = os.path.join(dirname,filename)
    if os.path.exists(path):
        return path
    else:
        print('no such template ' + filename)
        return ''

def get_ms_logg(m, m_err):
    ## note to self- log g is defined as log_10 of g in cm/s^2

    ## use main sequence scaling relation: mass proportional to radius
    r = m * R_sun
    dr = m_err * R_sun
    g = G.value * m * M_sun.value / r**2 * 1e2 ## in cm/s^2
    dg = g * np.sqrt((2*dr/r)**2 + (2*m_err/m)**2)
    return np.log10(g.value) 

def get_wd_logg(m,teff):
    ## WD radius nearly independent of mass
    m_logteff_to_logg = WD_models.interp_xy_z_func(x=model['mass_array'],
                                                  y=model['logteff'],
                                                  z=model['logg'],
                                                  interp_type='linear')
    logg = float(m_logteff_to_logg(m,np.log10(teff)))
    return logg

def get_distance(parallax,parallax_err):
    ## distance in meters calculated from parallax in mas
    return (1000/parallax) * pc.value , (1000/parallax**2)*parallax_err * pc.value

def inverse_square_coeff(logg, m, m_err, parallax, parallax_err):
    R , dR = get_radius(logg, m, m_err)
    D , dD = get_distance(parallax, parallax_err)
    c = (R / D)**2
    dc = np.sqrt((2*(R/D**2)*dR)**2 + (2*(R**2/D**3)*dD)**2)
    return c , dc

def model_MS(teff1, m1, m1_err, parallax, parallax_err, meta1 = -0.5):
    ## teff and logg are the parameters we wanna optimize (maybe alpha and meta too, one day)
    ## m (mass) and parallax should be given, to be used in the flux calculation
    logg1 = get_ms_logg(m1,m1_err)
    path1 = get_kurucz_spec_filepath(teff1, logg1, meta1)
    if len(path1) == 0: ## wrong path: change directory path / check the parameters indeed have a template
        return Table({'wavelength':[],'wavelength':[]}) 
    c1,dc1 = inverse_square_coeff(logg1,m1,m1_err,parallax,parallax_err)
    votable1 = parse(path1)
    table1 = Table.read(votable1.get_first_table())
    model_flux = c1 * table1['FLUX'].data * table1['WAVELENGTH'].data
    model_table = Table({'wavelength':table1['WAVELENGTH'].data,'flux':model_flux})
    return model_table

def model_WD(teff2, m2, m2_err, parallax, parallax_err):
    ## teff and logg are the parameters we wanna optimize
    ## m (mass) and parallax should be given, to be used in the flux calculation
    logg2 = get_wd_logg(m2,teff2)
    path2 = get_koester_spec_filepath(teff2, logg2)
    if len(path2) == 0: ## wrong path: change directory path / check the parameters indeed have a template
        return Table({'wavelength':[],'flux':[]}) 
    c2,dc2 = inverse_square_coeff(logg2,m2,m2_err,parallax,parallax_err)
    votable2 = parse(path2)
    table2 = Table.read(votable2.get_first_table())
    model_flux = c2 * table2['FLUX'].data * table2['WAVELENGTH'].data
    model_table = Table({'wavelength':table2['WAVELENGTH'].data,'flux':model_flux})
    return model_table

def plot_binary_ms(teff1, m1, m1_err, teff2, m2, m2_err, parallax, parallax_err, meta1 = 0, meta2  = 0):
    ms1 = model_MS(teff1, m1, m1_err, parallax, parallax_err, meta1)
    ms2 = model_MS(teff2, m2, m2_err, parallax, parallax_err, meta2)
    

    shared_wl = np.union1d(ms1['wavelength'].data,ms2['wavelength'].data)
    m1_interp = np.interp(shared_wl,ms1['wavelength'].data,ms1['flux'].data, left = 0, right = 0)
    m2_interp = np.interp(shared_wl,ms2['wavelength'].data,ms2['flux'].data, left = 0, right = 0)
    
    tot_flux = m1_interp + m2_interp
    
    fig , ax = plt.subplots(figsize=(figwidth_single, figwidth_single), tight_layout=True,dpi=300)
    ax.plot(ms1['wavelength'],ms1['flux'],linestyle='--',color='RoyalBlue',label=f'MS {int(teff1)}K',linewidth=1,alpha=0.8)
    ax.plot(ms2['wavelength'],ms2['flux'],linestyle='--',color='Crimson',label=f'MS {int(teff2)}K',linewidth=1,alpha=0.8)
    ax.plot(shared_wl,tot_flux,color='k',label=f'total flux',linewidth=1)
    ax.legend(loc = 'best')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel(r'$\lambda (\AA) $',fontdict={'size':12})
    ax.set_ylabel(r'$\lambda F_\lambda(erg/s/cm^2)$',fontdict={'size':12})
    ax.set_ylim(1e-20,10*np.max(ms1['flux']))
    ax.set_facecolor('0.7')
    ax.legend()
    ax.grid()
    fig.show()

def plot_triple_ms(teff1, m1, m1_err, teff2, m2, m2_err, parallax, parallax_err, meta1 = 0, meta2  = 0):
    ms1 = model_MS(teff1, m1, m1_err, parallax, parallax_err, meta1)
    ms2 = model_MS(teff2, m2 / 2, m2_err, parallax, parallax_err, meta2)

    shared_wl = np.union1d(ms1['wavelength'].data,ms2['wavelength'].data)
    m1_interp = np.interp(shared_wl,ms1['wavelength'].data,ms1['flux'].data, left = 0, right = 0)
    m23_interp = np.interp(shared_wl,ms2['wavelength'].data,ms2['flux'].data, left = 0, right = 0)
    
    tot_flux = m1_interp + 2 * m23_interp
    
    fig , ax = plt.subplots(figsize=(figwidth_single, figwidth_single), tight_layout=True,dpi=300)
    ax.plot(ms1['wavelength'],ms1['flux'],linestyle='--',color='RoyalBlue',label=f'MS {int(teff1)}K',linewidth=1,alpha=0.8)
    ax.plot(ms2['wavelength'],2 *ms2['flux'],linestyle='--',color='Crimson',label=f'double MS {int(teff2)}K',linewidth=1,alpha=0.8)
    ax.plot(m23_interp,tot_flux,color='k',label=f'total flux',linewidth=1)
    ax.legend(loc = 'best')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel(r'$\lambda (\AA) $',fontdict={'size':12})
    ax.set_ylabel(r'$\lambda F_\lambda(erg/s/cm^2)$',fontdict={'size':12})
    ax.set_facecolor('0.7')
    ax.set_ylim(1e-20,10*np.max(ms1['flux']))
    ax.legend()
    ax.grid()
    fig.show()

def plot_binary_wd(teff1, m1, m1_err, teff2, m2, m2_err, parallax, parallax_err, meta1 = 0, gratings = ['galex']):
    ms = model_MS(teff1, m1, m1_err, parallax, parallax_err, meta1)
    wd = model_WD(teff2, m2, m2_err, parallax, parallax_err)
    
    shared_wl = np.union1d(ms['wavelength'].data,wd['wavelength'].data)
    m_interp = np.interp(shared_wl,ms['wavelength'].data,ms['flux'].data, left = 0, right = 0)
    w_interp = np.interp(shared_wl,wd['wavelength'].data,wd['flux'].data, left = 0, right = 0)
    tot_flux = m_interp + w_interp

    fig , ax = plt.subplots(figsize=(figwidth_single, figwidth_single), tight_layout=True, dpi=300)
    ax.plot(ms['wavelength'],ms['flux'] ,linestyle='--',color='RoyalBlue',label=f'MS {int(teff1)}K',linewidth=1,alpha=0.8)
    ax.plot(wd['wavelength'],wd['flux'] ,linestyle='--',color='Crimson',label=f'WD {int(teff2)}K',linewidth=1,alpha=0.8)
    ax.plot(shared_wl,tot_flux,color='w',label=f'total flux',linewidth=1)
    if 800 in gratings:
        ax.vlines([815,1948],0,100,colors='Gold',lw=1,ls=':',label='COS G140L/800',alpha = 0.8)
    elif 1105 in gratings:
        ax.vlines([1118,2150],0,100,colors='Gold',lw=1,ls=':',label='COS G140L/1105',alpha = 0.8)
    if 2635 in gratings:
        ax.vlines([2435,2834],0,100,colors='HotPink',lw=1,ls=':',label='COS G230L/2635',alpha = 0.8)
    elif 'galex' in gratings:
        ax.vlines([1750,2800],0,100,colors='HotPink',lw=1,ls=':',label='GALEX NUV',alpha = 0.8)
    ax.legend(loc = 'best')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel(r'$\lambda (\AA) $',fontdict={'size':8})
    ax.set_ylabel(r'$\lambda F_\lambda(erg/s/cm^2)$',fontdict={'size':8})
    ax.set_facecolor('0.25')
    ax.set_ylim(1e-20,10*np.max(ms['flux']))
    ax.set_xlim(right = 5e3)
    ax.legend(fontsize=8)
    ax.tick_params(axis='both', which='both', labelsize=8)
    ax.grid()
    fig.show()
    return shared_wl, tot_flux / shared_wl ## wl in angstrom, flux in erg/s/cm^2/A

In [3]:
## NUV excess functions


def obs_nuv_flux(mag,av):
    # ext = G23(Rv = 3.1)
    f0 = gaia_galex_zp_table[gaia_galex_zp_table['band'] == 'GALEX.NUV']['f0'].data[0]
    m0 = gaia_galex_zp_table[gaia_galex_zp_table['band'] == 'GALEX.NUV']['m0'].data[0]
    f = 10**(-0.4 * (mag - m0)) * f0
    # f_dereddened = f / ext.extinguish(2303 * u.AA, Av = av)
    return f

def obs_fuv_flux(mag,av):
    # ext = G23(Rv = 3.1)
    f0 = gaia_galex_zp_table[gaia_galex_zp_table['band'] == 'GALEX.FUV']['f0'].data[0]
    m0 = gaia_galex_zp_table[gaia_galex_zp_table['band'] == 'GALEX.FUV']['m0'].data[0]
    f = 10**(-0.4 * (mag - m0)) * f0
    # f_dereddened = f / ext.extinguish(1528 * u.AA, Av = av)
    return f

def nuv_flux_to_mag(flux):
    f0 = gaia_galex_zp_table[gaia_galex_zp_table['band'] == 'GALEX.NUV']['f0'].data[0]
    m0 = gaia_galex_zp_table[gaia_galex_zp_table['band'] == 'GALEX.NUV']['m0'].data[0]
    return m0 - 2.5 * np.log10(flux / f0)

def fuv_flux_to_mag(flux):
    f0 = gaia_galex_zp_table[gaia_galex_zp_table['band'] == 'GALEX.FUV']['f0'].data[0]
    m0 = gaia_galex_zp_table[gaia_galex_zp_table['band'] == 'GALEX.FUV']['m0'].data[0]
    return m0 - 2.5 * np.log10(flux / f0)

def get_primary_nuv_lims(t_lo,t,t_hi,m_lo,m,m_hi,r_lo,r,r_hi,parallax,meta,av):
    ## get the NUV flux range of the primary star
    ms_hi = model_sed_ms(t_hi, m_hi, m_hi*0.1, parallax, parallax*0.1, meta1 = meta, r1 = r_hi)
    ms = model_sed_ms(t, m, m*0.1, parallax, parallax*0.1, meta1 = meta, r1 = r)
    ms_lo = model_sed_ms(t_lo, m_lo, m_lo*0.1, parallax, parallax*0.1, meta1 = meta, r1 = r_lo)
    _, ANUV_lo = extinction.get_Galex_extinction(av,0,t_lo)
    _, ANUV = extinction.get_Galex_extinction(av,0,t)
    _, ANUV_hi = extinction.get_Galex_extinction(av,0,t_hi)
    nuv_hi = ms_hi[ms_hi['filter'] == 'GALEX/GALEX.NUV']['flux'][0] / ms_hi[ms_hi['filter'] == 'GALEX/GALEX.NUV']['wavelength'][0]
    nuv = ms[ms['filter'] == 'GALEX/GALEX.NUV']['flux'][0] / ms[ms['filter'] == 'GALEX/GALEX.NUV']['wavelength'][0]
    nuv_lo = ms_lo[ms_lo['filter'] == 'GALEX/GALEX.NUV']['flux'][0] / ms_lo[ms_lo['filter'] == 'GALEX/GALEX.NUV']['wavelength'][0]
    nuv_hi = nuv_hi * 10**(-0.4 * ANUV_hi)
    nuv = nuv * 10**(-0.4 * ANUV)
    nuv_lo = nuv_lo * 10**(-0.4 * ANUV_lo)
    return nuv_lo, nuv, nuv_hi

def get_primary_fuv_lims(t_lo,t,t_hi,m_lo,m,m_hi,r_lo,r,r_hi,parallax,meta,av):
    ms_hi = model_sed_ms(t_hi, m_hi, m_hi*0.1, parallax, parallax*0.1, meta1 = meta, r1 = r_hi)
    ms = model_sed_ms(t, m, m*0.1, parallax, parallax*0.1, meta1 = meta, r1 = r)
    ms_lo = model_sed_ms(t_lo, m_lo, m_lo*0.1, parallax, parallax*0.1, meta1 = meta, r1 = r_lo)
    fuv_hi = ms_hi[ms_hi['filter'] == 'GALEX/GALEX.FUV']['flux'][0] / ms_hi[ms_hi['filter'] == 'GALEX/GALEX.FUV']['wavelength'][0]
    fuv = ms[ms['filter'] == 'GALEX/GALEX.FUV']['flux'][0] / ms[ms['filter'] == 'GALEX/GALEX.FUV']['wavelength'][0]
    fuv_lo = ms_lo[ms_lo['filter'] == 'GALEX/GALEX.FUV']['flux'][0] / ms_lo[ms_lo['filter'] == 'GALEX/GALEX.FUV']['wavelength'][0]
    AFUV_lo,_ = extinction.get_Galex_extinction(av,0,t_lo)
    AFUV,_ = extinction.get_Galex_extinction(av,0,t)
    AFUV_hi,_ = extinction.get_Galex_extinction(av,0,t_hi)
    fuv_hi = fuv_hi * 10**(-0.4 * AFUV_hi)
    fuv = fuv * 10**(-0.4 * AFUV)
    fuv_lo = fuv_lo * 10**(-0.4 * AFUV_lo)
    return fuv_lo, fuv, fuv_hi

def get_companion_RT_lims_fuv(obs_mag,fuv_lo,fuv,fuv_hi,m2,m2_err,parallax,av):
    obs_flux = obs_fuv_flux(obs_mag,av)

    mod_flux = np.zeros(len(ms_sed_teff_arr))

    for i,t in enumerate(ms_sed_teff_arr):
        r = get_radius(get_ms_logg(m2,t),m2,m2_err)[0] / R_sun.value
        if np.isnan(r):
            mod_flux[i] = np.nan
        else:
            wd = model_sed_wd(t, m2, m2_err, parallax, parallax*0.1)
            mod_flux[i] = wd[wd['filter'] == 'GALEX/GALEX.FUV']['flux'][0] / wd[wd['filter'] == 'GALEX/GALEX.FUV']['wavelength'][0]
            AFUV,_ = extinction.get_Galex_extinction(av,0,t)
            mod_flux[i] = mod_flux[i] * 10**(-0.4 * AFUV)
    excess_hi = obs_flux - fuv_lo
    excess_lo = obs_flux - fuv_hi
    excess = obs_flux - fuv
    
    t1 = wd_sed_teff_arr[np.nanargmin(np.abs(excess_lo - mod_flux))]
    t2 = wd_sed_teff_arr[np.nanargmin(np.abs(excess_hi - mod_flux))]
    t = wd_sed_teff_arr[np.nanargmin(np.abs(excess - mod_flux))]

    r1 = get_radius(get_wd_logg(m2,t1),m2,m2_err)[0] / R_sun.value
    r2 = get_radius(get_wd_logg(m2,t2),m2,m2_err)[0] / R_sun.value
    r = get_radius(get_wd_logg(m2,t),m2,m2_err)[0] / R_sun.value

    return [t1,t,t2],[r1,r,r2]

def get_companion_RT_lims_nuv(obs_mag,nuv_lo,nuv,nuv_hi,m2,m2_err,parallax,av):
    obs_flux = obs_nuv_flux(obs_mag,av)
    
    mod_flux = np.zeros(len(wd_sed_teff_arr))

    for i,t in enumerate(wd_sed_teff_arr):
        r = get_radius(get_wd_logg(m2,t),m2,m2_err)[0] / R_sun.value
        if np.isnan(r):
            mod_flux[i] = np.nan
        else:
            wd = model_sed_wd(t, m2, m2_err, parallax, parallax*0.1)
            mod_flux[i] = wd[wd['filter'] == 'GALEX/GALEX.NUV']['flux'][0] / wd[wd['filter'] == 'GALEX/GALEX.NUV']['wavelength'][0]
            _,ANUV = extinction.get_Galex_extinction(av,0,t)
            mod_flux[i] = mod_flux[i] * 10**(-0.4 * ANUV)
    excess_hi = obs_flux - nuv_lo
    excess_lo = obs_flux - nuv_hi
    excess = obs_flux - nuv
    
    t1 = wd_sed_teff_arr[np.nanargmin(np.abs(excess_lo - mod_flux))]
    t2 = wd_sed_teff_arr[np.nanargmin(np.abs(excess_hi - mod_flux))]
    t =  wd_sed_teff_arr[np.nanargmin(np.abs(excess - mod_flux))]

    r1 = get_radius(get_wd_logg(m2,t1),m2,m2_err)[0] / R_sun.value
    r2 = get_radius(get_wd_logg(m2,t2),m2,m2_err)[0] / R_sun.value
    r = get_radius(get_wd_logg(m2,t),m2,m2_err)[0] / R_sun.value
    
    return [t1,t,t2],[r1,r,r2]



# idx = 99
# i = np.where(sources['idx'] == idx)[0][0]

# for i in range(len(sources)):
#     idx = sources[i]['idx']

#     t_lo,t,t_hi = sources[i]['teff1'] - sources[i]['teff1_err'],sources[i]['teff1'] ,sources[i]['teff1'] + sources[i]['teff1_err']
#     m_lo,m,m_hi= sources[i]['m1'] - sources[i]['m1_err'], sources[i]['m1'] , sources[i]['m1'] + sources[i]['m1_err']
#     r_lo,r,r_hi = sources[i]['r1'] - sources[i]['r1_err'], sources[i]['r1'], sources[i]['r1'] + sources[i]['r1_err']
#     parallax = sources[i]['parallax']
#     meta = sources[i]['[Fe/H]']
#     av = sources[i]['Av']
#     m2 = sources[i]['m2']
#     m2_err = sources[i]['m2_err']
    
#     if not np.ma.is_masked(sources[i]['nuv_mag']):
#         obs_mag = sources[i]['nuv_mag']
#         nuv_lo,nuv,nuv_hi = get_primary_nuv_lims(t_lo,t,t_hi,m_lo,m,m_hi,r_lo,r,r_hi,parallax,meta,av)
#         nuv_tarr,nuv_rarr = get_companion_RT_lims_nuv(obs_mag,nuv_lo,nuv,nuv_hi,m2,m2_err,parallax,av)
#         print(idx,nuv_tarr,int(sources[i]['teff2_min']),sources[i]['m2'].round(2),sources[i]['cluster'])
#     if not np.ma.is_masked(sources[i]['fuv_mag']):
#         obs_mag = sources[i]['fuv_mag']
#         fuv_lo, fuv, fuv_hi = get_primary_fuv_lims(t_lo,t,t_hi,m_lo,m,m_hi,r_lo,r,r_hi,parallax,meta,av)
#         fuv_tarr,fuv_rarr = get_companion_RT_lims_fuv(obs_mag,fuv_lo,fuv,fuv_hi,m2,m2_err,parallax,av)
#         print(idx,fuv_tarr,int(sources[i]['teff2_min']),sources[i]['m2'].round(2),sources[i]['cluster'])


In [60]:
zp = 7.5e-16
print('idx cts/s texp')
for i in range(len(sources)):
    av = sources[i]['Av']
    anuv = extinction.get_Galex_extinction(av,0,sources[i]['teff2_min'])[0]
    m2 = sources[i]['m2']
    age = sources[i]['age']
    nuv = m_agecool_to_mag(m2,age*1e-3)
    parallax = sources[i]['parallax']
    nuv_apparent = nuv + 5 * np.log10(1000/parallax) - 5 + anuv
    c = obs_nuv_flux(nuv_apparent,av)/zp
    texp = 20**2 / c
    if texp < 1000 and np.ma.is_masked(sources[i]['nuv_mag']):
        print(sources[i]['idx'],c.round(2),int(texp))
    

idx cts/s texp
53 0.5 805
101 0.77 522
105 0.69 582
191 0.7 571
192 1.04 382
282 0.88 456


In [95]:
p1 = [53,118,174,175,180,249]
p3 = [16,33,68,71,80,121,236,270]
p4 = [114]

sources['priority'] = np.zeros(len(sources),dtype=int)

for i in range(len(sources)):
    idx = sources[i]['idx']
    if idx in p1:
        sources['priority'][i] = 1
    elif idx in p3:
        sources['priority'][i] = 3
    elif idx in p4:
        sources['priority'][i] = 4
    else:
        sources['priority'][i] = 2


In [41]:
# sources = sources[sources['idx'] == 99]

# sources['phot_g_mean_mag'] = sources['phot_g_mean_mag'].round(1)
# sources['ra'] = sources['ra'].round(5)
# sources['dec'] = sources['dec'].round(5)
# sources['distance'] = (1000/sources['parallax']).round(0)
# sources['distance_err'] = (1000/sources['parallax']**2 * sources['parallax_error']).round(0)
# sources['D'] = [f'{d}±{d_err}' for d,d_err in zip(sources['distance'],sources['distance_err'])]
# sources['nuv_mag'] = sources['nuv_mag'].round(1)
# sources['m1'] = sources['m1'].round(2)
# sources['m1_err'] = sources['m1_err'].round(2)
# sources['m2'] = sources['m2'].round(2)
# sources['m2_err'] = sources['m2_err'].round(2)
# sources['r1'] = sources['r1'].round(2)
# sources['r1_err'] = sources['r1_err'].round(2)
# sources['teff1'] = np.array(sources['teff1'].round(-1),dtype = int)
# sources['teff1_err'] = np.array(sources['teff1_err'].round(-1),dtype = int)
# sources['teff2'] = np.array(sources['teff2'].round(-1),dtype = int)
# sources['age'] = (10**sources['log_age_50'] * 1e-6).round(0)
# sources['M1'] = [f'{m:.2f}±{m_err:.2f}' for m,m_err in zip(sources['m1'],sources['m1_err'])]
# sources['R1'] = [f'{r:.2f}±{r_err:.2f}' for r,r_err in zip(sources['r1'],sources['r1_err'])]
# sources['M2'] = [f'{m:.2f}±{m_err:.2f}' for m,m_err in zip(sources['m2'],sources['m2_err'])]
# sources['T1'] = [f'{t}±{t_err}' for t,t_err in zip(sources['teff1'],sources['teff1_err'])]
# sources['[Fe/H]'] = sources['[Fe/H]'].round(2)
# sources['Av'] = sources['Av'].round(2)
# sources['age'] = sources['age'].round(0)   
# print(sources['T1','R1','M2','teff2','[Fe/H]','Av','age'].to_pandas().to_latex(index=False))

\begin{tabular}{lllrrrr}
\toprule
T1 & R1 & M2 & teff2 & mh_for_mass_interp & av_for_mass_interp & age \\
\midrule
5400±110 & 1.15±0.06 & 0.51±0.03 & 16990 & -0.050000 & 0.010000 & 162.000000 \\
\bottomrule
\end{tabular}



In [ ]:
sources = Table.read('../table_C.fits',format='fits')

# idx = 1
# i = np.where(sources['idx'] == idx)[0][0]
i = 5
idx = sources[i]['idx']

m1 = sources[i]['m1']
m1_err = sources[i]['m1_err']
m2 = sources[i]['m2']
if m2>1.25:
    m2 = sources[i]['m2'] - sources[i]['m2_err']
m2_err = sources[i]['m2_err']

parallax = sources[i]['parallax']
parallax_err = sources[i]['parallax_error']

teff1 = sources[i]['teff1']
# teff2 = sources[i]['teff2']

meta = sources[i]['[Fe/H]']

# wl, fl = plot_binary_wd(teff1,m1,m1_err,teff2,m2,m2_err,parallax,parallax_err,meta,gratings = [1105,'galex'])
plot_kurucz_vs_obs(sources,idx)
# plot_sed_ms_wd(teff1,m1,m1_err,teff2,m2,m2_err,parallax,parallax_err,meta)
# plot_binary_ms(8000,m1,m1_err,6000,m2,m2_err,parallax,parallax_err)
# plot_triple_ms(9000,m1,m1_err,3500,m2,m2_err,parallax,parallax_err)


# tbl = Table({'wavelength':wl,'flux':fl})
# cond = (tbl['wavelength'] > 925) & (tbl['wavelength'] < 80000)
# tbl = tbl[cond]
# tbl.write('../SBA28.dat',format='ascii',overwrite=True)